# Verify a FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Import Libraries

In [13]:
from requests import get, post
from json import dumps, loads
from pathlib import Path
# from datetime import datetime
# import pytz
from jose import jws  #python JWS package
from base64 import urlsafe_b64encode, b64decode
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.

In [14]:
local_file = True

bundle_id = "signed_object.json" # insert bundle or questionnaireresponse id here
# bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'
# bundle_id = 'Bundle-cdex-document-digital-sig-example.json'
# bundle_id = 'QuestionnaireResponse-cdex-questionnaireresponse-example4.json'

def fetch_payload():

  # url = "https://argopatientlist.aidbox.app/fhir/Bundle"
  # url = 'http://test.fhir.org/r4/Bundle'
  url = 'https://hl7.org/fhir/us/davinci-cdex'

  username = "basic"
  password = "secret"
  headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

  r = get(f'{url}/{bundle_id}', auth=(username, password), headers = headers)
  my_obj = r.json()
  # print("="*80)
  # print("STATUS: ",r.status_code)

  # print("="*80)
  # print("HEADERS:\n")
  # for k,v in r.headers.items():
  #     print(f'{k} = {v}')
  # print("="*80)
  # print("BODY:\n")
  # print(dumps(my_obj,indent=2))
  return(my_obj)

if local_file: 
  in_path = '/Users/ehaas/Documents/FHIR/davinci-ecdx/CDEX-Signatures/out_files'
  # in_path = '/Users/ehaas/Documents/FHIR/davinci-ecdx/output'
  # in_path = '/Users/ehaas/Downloads/'
  path = Path(in_path) / bundle_id
  my_obj =loads(path.read_text())
else:
  my_obj = fetch_payload()
print(dumps(my_obj,indent=2))

{
  "resourceType": "Bundle",
  "identifier": {
    "system": "urn:ietf:rfc:3986",
    "value": "urn:uuid:c173535e-135e-48e3-ab64-38bacc68dba8"
  },
  "type": "document",
  "timestamp": "2021-10-25T20:16:29-07:00",
  "entry": [
    {
      "fullUrl": "urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76",
      "resource": {
        "resourceType": "Composition",
        "id": "17a80a8d-4cf1-4deb-a1fd-2db1130e5f76",
        "text": {
          "status": "generated",
          "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h3>Medical Records Document</h3><p>This document, titled &quot;Active Conditions,&quot; was created on October 25, 2021, by Dr. John Hancock, who also legally attested to its accuracy on the same date. It summarizes the active medical conditions for a patient based on an emergency encounter on October 25, 2021. The document includes one section detailing an active condition: Type 2 Diabetes Mellitus.</p></div>"
        },
        "status": "final",
        "type": {
  

## Receiver/Verifier Steps

###  1. Remove the Signature from the Bundle or QR

In [15]:
if my_obj['resourceType'] == 'Bundle':
  recd_signature = my_obj.pop('signature', None) #for Bundles
elif my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
          recd_signature = my_obj_signature_ext['valueSignature']
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')
else:
  print('Not a Bundle or QuestionnaireResponse')

recd_signature

{'type': [{'system': 'urn:iso-astm:E1762-95:2013',
   'code': '1.2.840.10065.1.12.1.5',
   'display': 'Verification Signature'}],
 'when': '2020-10-23T04:54:56.048+00:00',
 'who': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v2-0203',
      'code': 'NPI'}]},
   'value': '9941339100'},
  'display': 'John Hancock, MD'},
 'onBehalfOf': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '1184932014'}},
 'targetFormat': 'application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document',
 'sigFormat': 'application/jose',
 'data': 'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNkltSm1ZbVV6WlRWak1EUTNNRFEwTkRVM05qVTBPREV4TXpreU9HUTFaakZsTkdVelpqSmxaV0lpTENKcmRIa2lPaUpTVXlJc0luTnBaMVFpT2lJeU1ESXdMVEV3TFRJelZEQTBPalUwT2pVMkxqQTBPQ3N3TURvd01DSXNJbk55UTIxeklqcGJleUpqYjIxdFNXUWlPbnNpWkdWell5STZJbFpsY21sbWFXTmhkR2x2YmlCVGFXZHVZWFIxY21VaUxDSnBaQ0k2SW5WeWJqcHZhV1E2TVM0eUxqZzBNQzR

### 2. Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS):

- Remove the id and meta elements if present before canonicalization

In [16]:
my_obj_id = my_obj.pop('id', None)
my_obj_meta = my_obj.pop('meta', None)
canonical_obj = canonicalize(my_obj)
canonical_obj, len(canonical_obj)

(b'{"entry":[{"fullUrl":"urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resource":{"attester":[{"mode":"legal","party":{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"},"time":"2021-10-25T20:16:29-07:00"}],"author":[{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"}],"date":"2021-10-25T20:16:29-07:00","encounter":{"display":"Example Encounter","reference":"urn:uuid:5ce5c83a-000f-47d2-941c-039358cc9112"},"id":"17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resourceType":"Composition","section":[{"entry":[{"reference":"urn:uuid:014a68ec-d691-49e0-b980-91b0d924e570"}],"title":"Active Condition 1"}],"status":"final","subject":{"display":"Example Patient","reference":"urn:uuid:970af6c9-5bbd-4067-b6c1-d9b2c823aece"},"text":{"div":"<div xmlns=\\"http://www.w3.org/1999/xhtml\\"><h3>Medical Records Document</h3><p>This document, titled &quot;Active Conditions,&quot; was created on October 25, 2021, by Dr. John Han

### 3. Transform canonicalize Bundle to a base64 format using the Base64-URL algorithm.

In [17]:
recd_b64_canonical_obj  = urlsafe_b64encode(canonical_obj).decode()
recd_b64_canonical_obj = recd_b64_canonical_obj.replace("=","")  #remove padding since decode package doesn't use them 
recd_b64_canonical_obj

'eyJlbnRyeSI6W3siZnVsbFVybCI6InVybjp1dWlkOjE3YTgwYThkLTRjZjEtNGRlYi1hMWZkLTJkYjExMzBlNWY3NiIsInJlc291cmNlIjp7ImF0dGVzdGVyIjpbeyJtb2RlIjoibGVnYWwiLCJwYXJ0eSI6eyJkaXNwbGF5IjoiRXhhbXBsZSBQcmFjdGl0aW9uZXIiLCJyZWZlcmVuY2UiOiJ1cm46dXVpZDowODIwYzE2ZC05MWRlLTRkZmEtYTNhNi1mMTQwYTUxNmE5YmMifSwidGltZSI6IjIwMjEtMTAtMjVUMjA6MTY6MjktMDc6MDAifV0sImF1dGhvciI6W3siZGlzcGxheSI6IkV4YW1wbGUgUHJhY3RpdGlvbmVyIiwicmVmZXJlbmNlIjoidXJuOnV1aWQ6MDgyMGMxNmQtOTFkZS00ZGZhLWEzYTYtZjE0MGE1MTZhOWJjIn1dLCJkYXRlIjoiMjAyMS0xMC0yNVQyMDoxNjoyOS0wNzowMCIsImVuY291bnRlciI6eyJkaXNwbGF5IjoiRXhhbXBsZSBFbmNvdW50ZXIiLCJyZWZlcmVuY2UiOiJ1cm46dXVpZDo1Y2U1YzgzYS0wMDBmLTQ3ZDItOTQxYy0wMzkzNThjYzkxMTIifSwiaWQiOiIxN2E4MGE4ZC00Y2YxLTRkZWItYTFmZC0yZGIxMTMwZTVmNzYiLCJyZXNvdXJjZVR5cGUiOiJDb21wb3NpdGlvbiIsInNlY3Rpb24iOlt7ImVudHJ5IjpbeyJyZWZlcmVuY2UiOiJ1cm46dXVpZDowMTRhNjhlYy1kNjkxLTQ5ZTAtYjk4MC05MWIwZDkyNGU1NzAifV0sInRpdGxlIjoiQWN0aXZlIENvbmRpdGlvbiAxIn1dLCJzdGF0dXMiOiJmaW5hbCIsInN1YmplY3QiOnsiZGlzcGxheSI6IkV4YW1wbGUgUGF0aWVudCIsInJlZmVyZW5jZSI

### 4. Get the base64 encoded JWS  from the `Bundle.signature.data`  element

In [18]:
recd_b64_jws = recd_signature['data']
recd_b64_jws

'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNkltSm1ZbVV6WlRWak1EUTNNRFEwTkRVM05qVTBPREV4TXpreU9HUTFaakZsTkdVelpqSmxaV0lpTENKcmRIa2lPaUpTVXlJc0luTnBaMVFpT2lJeU1ESXdMVEV3TFRJelZEQTBPalUwT2pVMkxqQTBPQ3N3TURvd01DSXNJbk55UTIxeklqcGJleUpqYjIxdFNXUWlPbnNpWkdWell5STZJbFpsY21sbWFXTmhkR2x2YmlCVGFXZHVZWFIxY21VaUxDSnBaQ0k2SW5WeWJqcHZhV1E2TVM0eUxqZzBNQzR4TURBMk5TNHhMakV5TGpFdU5TSjlMQ0pqYjIxdFVYVmhiSE1pT2xzaVZtVnlhV1pwWTJGMGFXOXVJRzltSUcxbFpHbGpZV3dnY21WamIzSmtJR2x1ZEdWbmNtbDBlU0pkZlYwc0luUjVjQ0k2SWtwWFZDSXNJbmcxWXlJNld5Sk5TVWxHVm5wRFEwRTNLMmRCZDBsQ1FXZEpWVUp1TkROR05FOHhNVEI2Vms1SVVGVjBRbTV1V0dZek0wWlJkM2RFVVZsS1MyOWFTV2gyWTA1QlVVVk1RbEZCZDJkYVZYaERla0ZLUW1kT1ZrSkJXVlJCYkZaVVRWSk5kMFZSV1VSV1VWRkpSRUZ3UkZsWGVIQmFiVGw1WW0xc2FFMVNTWGRGUVZsRVZsRlJTRVJCYkZSWldGWjZXVmQ0Y0dSSE9IaElWRUZpUW1kT1ZrSkJiMDFHUlZZMFdWY3hkMkpIVldkVU0wcHVXVmMxY0dWdFJqQmhWemwxVFZKcmQwWjNXVVJXVVZGRVJFSkNTMkl5YUhWSlJXaG9ZbTFPZGxreWMzTkpSVEZGVFZOTmQwbFJXVXBMYjFwSmFIWmpUa0ZSYTBKR2FGSnhZVWRHZFZreU9XcGhNRUpzWlVkR2RHTkhlR3hNYlRsNVducEJaVVozTUhsT1Z

### 5. Base64 decode the encoded JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [19]:
labels = ['header', 'payload', 'signature']
recd_jws = b64decode(recd_b64_jws.encode()).decode()
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()
print(recd_b64_canonical_obj == j)

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6ImJmYmUzZTVjMDQ3MDQ0NDU3NjU0ODExMzkyOGQ1ZjFlNGUzZjJlZWIiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInNyQ21zIjpbeyJjb21tSWQiOnsiZGVzYyI6IlZlcmlmaWNhdGlvbiBTaWduYXR1cmUiLCJpZCI6InVybjpvaWQ6MS4yLjg0MC4xMDA2NS4xLjEyLjEuNSJ9LCJjb21tUXVhbHMiOlsiVmVyaWZpY2F0aW9uIG9mIG1lZGljYWwgcmVjb3JkIGludGVncml0eSJdfV0sInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGVnpDQ0E3K2dBd0lCQWdJVUJuNDNGNE8xMTB6Vk5IUFV0Qm5uWGYzM0ZRd3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpVeU16RXlNemxhRncweU56QTJNVFV5TXpFeU16bGFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5y

### 6. reattach the payload - the base64 encoded Bundle or QR - into the JWS payload element. 

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [20]:
split_sig = recd_jws.split('.')
split_sig[1] = recd_b64_canonical_obj
recd_jws = '.'.join(split_sig)
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()
print(recd_b64_canonical_obj == j)

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6ImJmYmUzZTVjMDQ3MDQ0NDU3NjU0ODExMzkyOGQ1ZjFlNGUzZjJlZWIiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInNyQ21zIjpbeyJjb21tSWQiOnsiZGVzYyI6IlZlcmlmaWNhdGlvbiBTaWduYXR1cmUiLCJpZCI6InVybjpvaWQ6MS4yLjg0MC4xMDA2NS4xLjEyLjEuNSJ9LCJjb21tUXVhbHMiOlsiVmVyaWZpY2F0aW9uIG9mIG1lZGljYWwgcmVjb3JkIGludGVncml0eSJdfV0sInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGVnpDQ0E3K2dBd0lCQWdJVUJuNDNGNE8xMTB6Vk5IUFV0Qm5uWGYzM0ZRd3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpVeU16RXlNemxhRncweU56QTJNVFV5TXpFeU16bGFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5y

### 7. Obtain public Key from the first certificate in JWS header `"x5c"` key

- base64 decode the key value
- Verify Issuer, Validity Dates, Subject,and KeyUsage of certificate ( with the source )
- Get the “Subject Public Key Info” from the cert to verify the signature

In [21]:
recd_header = jws.get_unverified_header(recd_jws)
recd_header

{'alg': 'RS256',
 'kid': 'bfbe3e5c0470444576548113928d5f1e4e3f2eeb',
 'kty': 'RS',
 'sigT': '2020-10-23T04:54:56.048+00:00',
 'srCms': [{'commId': {'desc': 'Verification Signature',
    'id': 'urn:oid:1.2.840.10065.1.12.1.5'},
   'commQuals': ['Verification of medical record integrity']}],
 'typ': 'JWT',
 'x5c': ['MIIFVzCCA7+gAwIBAgIUBn43F4O110zVNHPUtBnnXf33FQwwDQYJKoZIhvcNAQELBQAwgZUxCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApDYWxpZm9ybmlhMRIwEAYDVQQHDAlTYXVzYWxpdG8xHTAbBgNVBAoMFEV4YW1wbGUgT3JnYW5pemF0aW9uMRkwFwYDVQQDDBBKb2huIEhhbmNvY2ssIE1EMSMwIQYJKoZIhvcNAQkBFhRqaGFuY29ja0BleGFtcGxlLm9yZzAeFw0yNTA2MjUyMzEyMzlaFw0yNzA2MTUyMzEyMzlaMIGVMQswCQYDVQQGEwJVUzETMBEGA1UECAwKQ2FsaWZvcm5pYTESMBAGA1UEBwwJU2F1c2FsaXRvMR0wGwYDVQQKDBRFeGFtcGxlIE9yZ2FuaXphdGlvbjEZMBcGA1UEAwwQSm9obiBIYW5jb2NrLCBNRDEjMCEGCSqGSIb3DQEJARYUamhhbmNvY2tAZXhhbXBsZS5vcmcwggGiMA0GCSqGSIb3DQEBAQUAA4IBjwAwggGKAoIBgQCcXmvX60GA5G+Dl4iRn9TS/wTF1FTH9RmrP29G6XSOuVDEgGZwUTJI/OkRLPj+JUKy/kMY3Ym41k3JRr8NrJ7Ucjf3Te2Y0zmRMfGKO2X7p01Id8rGhnbsTkWjszcc

### 8. Inspect the x509 certificate for the

- Issuer
- Subject
- Key Usage
- Validaty dates
- SAN

Define the location of the Document Signing Certificate for comparison to verify 

 - See the Jupyter file [Create_Cert.ipynb]() for how to generate your own self-signed certificate.

In [22]:
recd_cert = b64decode(recd_header['x5c'][0])
with open('recd_cert.der', 'wb') as f:
    f.write(recd_cert)
certificate_path = Path('example_org_cert') # update this to your folder
print(f'Source certificate for comparison is at ~/{certificate_path}')
!openssl x509 -in recd_cert.der -inform DER -text

Source certificate for comparison is at ~/example_org_cert
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            06:7e:37:17:83:b5:d7:4c:d5:34:73:d4:b4:19:e7:5d:fd:f7:15:0c
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, ST=California, L=Sausalito, O=Example Organization, CN=John Hancock, MD, emailAddress=jhancock@example.org
        Validity
            Not Before: Jun 25 23:12:39 2025 GMT
            Not After : Jun 15 23:12:39 2027 GMT
        Subject: C=US, ST=California, L=Sausalito, O=Example Organization, CN=John Hancock, MD, emailAddress=jhancock@example.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (3072 bit)
                Modulus:
                    00:9c:5e:6b:d7:eb:41:80:e4:6f:83:97:88:91:9f:
                    d4:d2:ff:04:c5:d4:54:c7:f5:19:ab:3f:6f:46:e9:
                    74:8e:b9:50:c4:80:66:70:51:32:48:fc:e9:11:2c:
                    f8:fe:25

### 9. Verify Signature using the public key or the X.509 Certificate

Alternatively:
1. visit https://jwt.io.
1. At the top of the page, select "RS256" for the algorithm.
1. Paste the JWS value printed below into the “Encoded” field.
1. The plaintext JWT will be displayed in the “Decoded:Payload” field.
1. The X509 Cert above will appear in the "Verify Signature" box.
1. If verified, a “Signature Verified” message will appear  in the bottom left hand corner.

In [23]:
recd_jws

'eyJhbGciOiJSUzI1NiIsImtpZCI6ImJmYmUzZTVjMDQ3MDQ0NDU3NjU0ODExMzkyOGQ1ZjFlNGUzZjJlZWIiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInNyQ21zIjpbeyJjb21tSWQiOnsiZGVzYyI6IlZlcmlmaWNhdGlvbiBTaWduYXR1cmUiLCJpZCI6InVybjpvaWQ6MS4yLjg0MC4xMDA2NS4xLjEyLjEuNSJ9LCJjb21tUXVhbHMiOlsiVmVyaWZpY2F0aW9uIG9mIG1lZGljYWwgcmVjb3JkIGludGVncml0eSJdfV0sInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGVnpDQ0E3K2dBd0lCQWdJVUJuNDNGNE8xMTB6Vk5IUFV0Qm5uWGYzM0ZRd3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpVeU16RXlNemxhRncweU56QTJNVFV5TXpFeU16bGFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJ

In [24]:
!openssl x509 -in recd_cert.der -inform DER -pubkey -noout > recd-public-key.pem
with open('recd-public-key.pem') as f:
    pem_public = f.read()
try:
    verify = jws.verify(recd_jws, pem_public , algorithms=['RS256'])
    print('#     #                                                ### ')
    print('#     #  ######  #####   #  ######  #  ######  #####   ### ')
    print('#     #  #       #    #  #  #       #  #       #    #  ### ')
    print('#     #  #####   #    #  #  #####   #  #####   #    #   #  ')
    print(' #   #   #       #####   #  #       #  #       #    #      ')
    print('  # #    #       #   #   #  #       #  #       #    #  ### ')
    print('   #     ######  #    #  #  #       #  ######  #####   ### ')
    print('                                                           ')

except Exception as e:
    print('#    #   ####   #####      #    #  ######  #####   #  ######  #  ######  #####       ###          #   ')
    print('##   #  #    #    #        #    #  #       #    #  #  #       #  #       #    #       #          #    ')
    print('# #  #  #    #    #        #    #  #####   #    #  #  #####   #  #####   #    #           #####  #    ')
    print('#  # #  #    #    #        #    #  #       #####   #  #       #  #       #    #       #          #    ')
    print('#   ##  #    #    #         #  #   #       #   #   #  #       #  #       #    #      ###          #   ')
    print('#    #   ####     #          ##    ######  #    #  #  #       #  ######  #####        #            ## ')
    print('                ')
    print(f"not verified: {e}")

#     #                                                ### 
#     #  ######  #####   #  ######  #  ######  #####   ### 
#     #  #       #    #  #  #       #  #       #    #  ### 
#     #  #####   #    #  #  #####   #  #####   #    #   #  
 #   #   #       #####   #  #       #  #       #    #      
  # #    #       #   #   #  #       #  #       #    #  ### 
   #     ######  #    #  #  #       #  ######  #####   ### 
                                                           
